<a href="https://colab.research.google.com/github/bardiaHSZD/TrendPrediction/blob/main/Dynamic_Mode_Decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment Installation**

In [ ]:
!apt install octave
!apt-get update --fix-missing
!apt install octave

# **Dynamic Mode Decomposition:** Test

In [4]:
%%writefile dmd_test_01.m

clear all;

%Quick Test of DMD with control.

%Data collection

A = [1.5 0;0 0.1];
x0 = [4;7];
K = [-1];
m = 20;
DataX = x0;
DataU = [0];

B = [1;0];

for j = 1 : m 
   
  DataX(:,j+1) = A * (DataX(:,j)) + B.*(K*DataX(:,j));
  DataU(:,j) = K .* DataX(1,j);
end

%Data matrices
X   = DataX(:,1:end-1);
Xp  = DataX(:,2:end);
Ups = DataU;

%SVD
[U,Sig,V] = svd(X,'econ');

thresh = 1e-10;
r = length(find(diag(Sig)>thresh));

U    = U(:,1:r);
Sig  = Sig(1:r,1:r);
V    = V(:,1:r);

%DMD

A_DMD  = Xp*V*inv(Sig)*U'

%DMDc - B is known 

A_DMDc = (Xp - B*Ups)*V*inv(Sig)*U'

% Extra
[W,D] = eig(A_DMDc);

Phi = Xp * V * inv(Sig) * U'*U * W

% DMD Spectra
t = linspace (0,m,m);
dt = t(2) - t(1);
lambda = diag(D);
omega = log(lambda)/dt;

% Compute DMD Solution
x0 = [4;7];
K = [-1];
m = 20;
DataXdmd = x0;
DataUdmd = [0];

B = [1.0;0.0];

for j = 1 : m   
  DataXdmd(:,j+1) = A_DMDc * (DataXdmd(:,j)) + B.*(K*DataXdmd(:,j));
  DataUdmd(:,j) = K .* DataXdmd(1,j);
end

%Data matrices
XDMD   = DataXdmd(:,1:end-1)
Ups = DataUdmd;

Writing dmd_test_01.m


In [5]:
!octave dmd_test_01.m # -W : no window system

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
A_DMD =

   5.0000e-01  -2.1751e-17
  -7.4096e-17   1.0000e-01

A_DMDc =

   1.5000e+00   2.2711e-16
  -7.4096e-17   1.0000e-01

Phi =

  -0.266675  -0.422947
  -0.084589   0.053335

XDMD =

 Columns 1 through 6:

   4.0000e+00   2.0000e+00   1.0000e+00   5.0000e-01   2.5000e-01   1.2500e-01
   7.0000e+00   7.0000e-01   7.0000e-02   7.0000e-03   7.0000e-04   7.0000e-05

 Columns 7 through 12:

   6.2500e-02   3.1250e-02   1.5625e-02   7.8125e-03   3.9063e-03   1.9531e-03
   7.0000e-06   7.0000e-07   7.0000e-08   7.0000e-09   7.0000e-10   7.0000e-11

 Columns 13 through 18:

   9.7656e-04   4.8828e-04   2.4414e-04   1.2207e-04   6.1035e-05   3.0518e-05
   7.0000e-12   7.0000e-13   7.0000e-14   7.0000e-15   6.9999e-16   6.9994e-17

 Columns 19 and 20:

   1.5259e-05   7.6294e-06
   6.9972e-18   6.9859e-19



# **Dynamic Mode Decomposition:** Algorithm

In [19]:
%%writefile dmd_full.m

A = [1.5 0;0 0.1];
x0 = [4;7];
K = [-1];
m = 20;
DataX = x0;
DataU = [0];

B = [1;0];

for j = 1 : m 
   
  DataX(:,j+1) = A * (DataX(:,j)) + B.*(K*DataX(:,j));
  DataU(:,j) = K .* DataX(1,j);
end

%Data matrices
X   = DataX(:,1:end-1);
Xp  = DataX(:,2:end);
Ups = DataU;
# --------------------------------------------------------
#X   = StateData(:,1:end-1);
#Xp  = StateData(:,2:end);
#Ups = InputData(:,1:end-1);

## =======================================================

Omega = [X;Ups];

[U,Sig,V] = svd(Omega,'econ');

thresh = 1e-10;
rtil = length(find(diag(Sig)>thresh));

Util    = U(:,1:rtil);
Sigtil  = Sig(1:rtil,1:rtil);
Vtil    = V(:,1:rtil); 

[U,Sig,V] = svd(Xp,'econ');

thresh = 1e-20;
r = length(find(diag(Sig)>thresh));

Uhat    = U(:,1:r); 
Sighat  = Sig(1:r,1:r);
Vbar    = V(:,1:r); 

n = size(X,1); 
q = size(Ups,1);
U_1 = Util(1:n,:);
U_2 = Util(n+q:n+q,:);

approxA = Uhat'*(Xp)*Vtil*inv(Sigtil)*U_1'*Uhat
approxB = Uhat'*(Xp)*Vtil*inv(Sigtil)*U_2'

[W,D] = eig(approxA);

Phi = Xp * Vtil * inv(Sigtil) * U_1'*Uhat * W

% Testing

A = approxA;
x0 = [4;7];
K = [-1];
m = 20;
DataX = x0;
DataU = [0];

B = approxB;

for j = 1 : m 
   
  DataX(:,j+1) = A * (DataX(:,j)) + B.*(K*DataX(:,j));
  DataU(:,j) = K .* DataX(1,j);
end

%Data matrices
X   = DataX(:,1:end-1)
Xp  = DataX(:,2:end);
Ups = DataU;

Overwriting dmd_full.m


In [20]:
!octave dmd_full.m # -W : no window system

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
approxA =

   0.239052   0.039018
   0.039018   0.110948

approxB =

   0.240703
   0.067541

Phi =

  -2.5000e-01   2.3699e-17
   2.8402e-17   1.0000e-01

X =

 Columns 1 through 6:

   4.0000e+00   2.6652e-01   1.7505e-02   1.1558e-03   7.6164e-05   5.0227e-06
   7.0000e+00   4.5992e-01   3.0363e-02   2.0010e-03   1.3195e-04   8.6994e-06

 Columns 7 through 12:

   3.3113e-07   2.1833e-08   1.4395e-09   9.4911e-11   6.2577e-12   4.1259e-13
   5.7359e-07   3.7818e-08   2.4934e-09   1.6440e-10   1.0839e-11   7.1467e-13

 Columns 13 through 18:

   2.7203e-14   1.7936e-15   1.1826e-16   7.7969e-18   5.1407e-19   3.3894e-20
   4.7120e-14   3.1067e-15   2.0484e-16   1.3505e-17   8.9045e-19   5.8710e-20

 Columns 19 and 20:

   2.2347e-21   1.4734e-22
   3.8709e-21   2.5522e-22

